## Задание 1: Циклотомические классы

In [1]:
def cyclotomic_classes(p: int, n: int) -> dict[tuple, list[int]]:
    vs = range(n)
    visited = {v: False for v in vs}
    ptr = 0
    classes = {}
    while ptr < n:
        if not visited[ptr]:
            cls = [ptr]
            cur = ptr
            visited[cur] = True
            while True:
                cur = cur * p % n
                visited[cur] = True
                if cur == ptr:
                    break
                else:
                    cls.append(cur)
            classes[(ptr,)] = cls
        ptr += 1
    return classes

In [2]:
p = 2
n = 23

classes = cyclotomic_classes(p, n)
print('Cyclotomic classes:')
print(classes)

Cyclotomic classes:
{(0,): [0], (1,): [1, 2, 4, 8, 16, 9, 18, 13, 3, 6, 12], (5,): [5, 10, 20, 17, 11, 22, 21, 19, 15, 7, 14]}


## Задание 2: неприводимые БЧХ коды

In [3]:
from itertools import combinations, chain
from pprint import pprint
from copy import deepcopy
import pandas as pd


def longest_consecutive_subsequence(seq: list[int]) -> int:
    lookup = set(seq)
    longest = 0
    for num in seq:
        if num - 1 not in lookup:
            cur_num = num
            cur_len = 1
            while cur_num + 1 in lookup:
                cur_num += 1
                cur_len += 1
            longest = max(longest, cur_len)
    return longest

def bch_codes(classes: dict[tuple, list[int]], copy=False):
    if copy:
        results = deepcopy(classes)
    else:
        results = {}
    for i in range(2, len(classes) + 1):
        for combo in combinations(classes.items(), i):
            labels, values = zip(*combo)
            labels = tuple(sorted(chain.from_iterable(labels)))
            values = sorted(chain.from_iterable(values))
            results[labels] = values
    return results

def codes_info(codes: dict[tuple, list[int]], n: int):
    info = []
    for labels, degs in codes.items():
        info.append({
            'Degrees': degs,
            'g(x)': '(' + '.'.join(f'M{l}' for l in labels) + ')(x)',
            'k': n - len(degs),
            'Distance': longest_consecutive_subsequence(degs) + 1,
        })
    return info

In [4]:
codes = bch_codes(classes, True)
pprint(codes)
info = codes_info(codes, n)
pprint(info)

{(0,): [0],
 (0, 1): [0, 1, 2, 3, 4, 6, 8, 9, 12, 13, 16, 18],
 (0, 1, 5): [0,
             1,
             2,
             3,
             4,
             5,
             6,
             7,
             8,
             9,
             10,
             11,
             12,
             13,
             14,
             15,
             16,
             17,
             18,
             19,
             20,
             21,
             22],
 (0, 5): [0, 5, 7, 10, 11, 14, 15, 17, 19, 20, 21, 22],
 (1,): [1, 2, 4, 8, 16, 9, 18, 13, 3, 6, 12],
 (1, 5): [1,
          2,
          3,
          4,
          5,
          6,
          7,
          8,
          9,
          10,
          11,
          12,
          13,
          14,
          15,
          16,
          17,
          18,
          19,
          20,
          21,
          22],
 (5,): [5, 10, 20, 17, 11, 22, 21, 19, 15, 7, 14]}
[{'Degrees': [0], 'Distance': 2, 'g(x)': '(M0)(x)', 'k': 22},
 {'Degrees': [1, 2, 4, 8, 16, 9, 18, 13,

In [5]:
%%capture
import import_ipynb
import sys
import os
from min_polys_and_orders import register_self_feed

In [6]:
import numpy as np

In [7]:
%%capture
n = 4
q = 2
mod = q ** n - 1
begin_state = [1, 0, 0, 0]
denominator = [1, 1, 0, 0, 1]
msg = [0]

alphas = np.array([list(state) for state in register_self_feed(q, begin_state, denominator, 14)])
A = dict(enumerate(alphas))

In [8]:
A

{0: array([1, 0, 0, 0]),
 1: array([0, 1, 0, 0]),
 2: array([0, 0, 1, 0]),
 3: array([0, 0, 0, 1]),
 4: array([1, 1, 0, 0]),
 5: array([0, 1, 1, 0]),
 6: array([0, 0, 1, 1]),
 7: array([1, 1, 0, 1]),
 8: array([1, 0, 1, 0]),
 9: array([0, 1, 0, 1]),
 10: array([1, 1, 1, 0]),
 11: array([0, 1, 1, 1]),
 12: array([1, 1, 1, 1]),
 13: array([1, 0, 1, 1]),
 14: array([1, 0, 0, 1])}

In [ ]:
def A_lookup(A: dict[int, np.ndarray], mod: int, q: int):
    At = {i: tuple(alpha) for i, alpha in A.items()}
    A_rev = {alpha: i for i, alpha in At.items()}
    lookup = {
        beta: {alpha: -1 for alpha in At.values()}
        for beta in At.values()
    }
    for i in range(mod):
        for j in range(i + 1, mod):
            mul = (A[i] + A[j]) % q
            lookup[At[i], At[j]] = A_rev[tuple(mul)]
            lookup[At[j], At[i]] = lookup[At[i], At[j]]
    return lookup, At

def print_L_formatted():
    pass

def rs_syndromes(A: dict[int, np.ndarray], v: list[int], mod: int, q: int):
    syndromes = {}
    L, At = A_lookup(A, mod, q)
    
    for j in range(mod):
        syndromes[j] = np.sum([A[L[At[j]][At[i]]] for i in range(mod)], axis=0) % q
        # print(np.sum([np.multiply(A[j], A[i]) for i in range(mod)], axis=0))
        # syndromes[j] = np.sum([np.multiply(A[j], A[i]) for i in range(mod)], axis=0) % q
        # break
    return syndromes

In [18]:
v = [1, 3, 7, 12, 13, 14, 9, 5, 0, 10, 8, 1, 10, 13, 2]
syndromes = rs_syndromes(A, v, mod, q)
pprint(syndromes)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

## Задание 3: вектор ошибок